In [1]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import math
from PIL import Image
import random

In [2]:
from utils import movie_images_to_dict

In [3]:
trainning_folder="trainning_images"
test_folder=""

In [4]:
trainning_data_set=[]
test_data_set=[]
# Loop through trainning images
for index in range(3):
    trainning_data_set.append(pd.DataFrame(movie_images_to_dict.get_images_to_dict(f'/media/Data/backwards_detection/trainning_images/{index}/')).to_numpy())
# Loop through test images
for index in range(1):
    test_data_set.append(pd.DataFrame(movie_images_to_dict.get_images_to_dict(f'/media/Data/backwards_detection/test_images/{index}/')).to_numpy())

In [5]:
pd.DataFrame(test_data_set[0])

,0,1,2,3
0,00000001.png,/media/Data/backwards_detection/test_images/0/,800,1280
1,00000002.png,/media/Data/backwards_detection/test_images/0/,800,1280
2,00000003.png,/media/Data/backwards_detection/test_images/0/,800,1280
3,00000004.png,/media/Data/backwards_detection/test_images/0/,800,1280
4,00000005.png,/media/Data/backwards_detection/test_images/0/,800,1280
...,...,...,...,...
149727,00149728.png,/media/Data/backwards_detection/test_images/0/,800,1280
149728,00149729.png,/media/Data/backwards_detection/test_images/0/,800,1280
149729,00149730.png,/media/Data/backwards_detection/test_images/0/,800,1280
149730,00149731.png,/media/Data/backwards_detection/test_images/0/,800,1280


In [6]:
from typing import List, Tuple, Union
def shape(ndarray: Union[List, float]) -> Tuple[int, ...]:
    if isinstance(ndarray, list):
        # More dimensions, so make a recursive call
        outermost_size = len(ndarray)
        row_shape = shape(ndarray[0])
        return (outermost_size, *row_shape)
    else:
        # No more dimensions, so we're done
        return ()


In [7]:
# conversion methods 
# will generate a array of x frames
def every_x_frame(data,x=5):
    size=data.shape[0]
    left_over=size%x
    n_times=math.floor(size/x)
    data=data[:-left_over,:]
    split=np.split(data,n_times)
    return split

def negative_frames(data,x):
    data_to_be_shuffled=data.copy()
    
    np.random.shuffle(data_to_be_shuffled)
    random=every_x_frame(data_to_be_shuffled,x)
    flip=every_x_frame(np.flip(data,axis=0),x)
    return flip+random

# generates positive and negative tests for x frames
def gen_every_x_frames(data,x=5):
    data=data.copy()
    positive=every_x_frame(data,x)
    negative=negative_frames(data,x)
    return [positive,negative]

# generates positive and negative tests for x frames
def gen_every_x_frames_skip(data,step=2,x=5):
    data=data.copy()
    data=data[::step]
    return gen_every_x_frames(data,x)

In [8]:
def trainning_data_set_gen():
    data_set_pos=[]
    data_set_neg=[]
    for trainning in trainning_data_set:
        # generate cases for 5 images and a skip of 2
        positive, negative=gen_every_x_frames_skip(trainning)
        data_set_pos=data_set_pos+positive
        data_set_neg=data_set_neg+negative
    return data_set_pos,data_set_neg
def test_data_set_gen():
    data_set_pos=[]
    data_set_neg=[]
    for test in test_data_set:
        # generate cases for 5 images and a skip of 2
        positive, negative=gen_every_x_frames_skip(test)
        data_set_pos=data_set_pos+positive
        data_set_neg=data_set_neg+negative
    return data_set_pos,data_set_neg

def pick_x_amount(data,x):
    return random.sample(data,x)

def open_images_prep_keras(posData, negData, limit, random=True):
    # take positive and negative data and limit
    if random:
        posData=pick_x_amount(posData,limit)
        negData=pick_x_amount(negData,limit)
    else:
        posData=posData[0:limit]
        negData=negData[0:limit]
    # image_paths Y
    data_set=[]
    for pos_images in posData:
        image_paths=[]
        for image in pos_images:
            image_paths.append(image[1]+image[0])
        xy=[image_paths,1]
        data_set.append(xy)
    for neg_images in negData:
        image_paths=[]
        for image in neg_images:
            image_paths.append(image[1]+image[0])
        xy=[image_paths,0]
        data_set.append(xy)
    data_set=np.array(data_set)
    np.random.shuffle(data_set)
    for data in data_set:
        image_data=[]
        for image_path in data[0]:
            image_data.append(np.array(Image.open(image_path))/255)
        yield np.array([np.array(image_data),np.array(data[1])])

def trainning_images_gen():
    keras_trainning_data_set_pos, keras_trainning_data_set_neg= trainning_data_set_gen()
    return open_images_prep_keras(keras_trainning_data_set_pos,keras_trainning_data_set_neg,1)
trainning_images_gen()

def test_images_gen():
    keras_test_data_set_pos, keras_test_data_set_neg= test_data_set_gen()
    return open_images_prep_keras(keras_test_data_set_pos,keras_test_data_set_neg,1)

In [26]:
# convert input to output
def start_of_cnn():
    image_input=tf.keras.layers.Input(shape=(800,1920,3))
    x=tf.keras.layers.Conv2D(filters=50,kernel_size=[3,3],activation='relu',padding='valid')(image_input)
    x=tf.keras.layers.MaxPooling2D()(x)
    x=tf.keras.layers.Dropout(.1)(x)
    x=tf.keras.layers.Conv2D(filters=25,kernel_size=[3,3],activation='relu',padding='valid')(x)
    x=tf.keras.layers.MaxPooling2D()(x)
    x=tf.keras.layers.Dropout(.1)(x)
    x=tf.keras.layers.Conv2D(filters=25,kernel_size=[3,3],activation='relu',padding='valid')(x)
    x=tf.keras.layers.MaxPooling2D()(x)
    x=tf.keras.layers.Dropout(.1)(x)
    x=tf.keras.layers.Conv2D(filters=25,kernel_size=[3,3],activation='relu',padding='valid')(x)
    x=tf.keras.layers.MaxPooling2D()(x)
    x=tf.keras.layers.Dropout(.1)(x)
    # x=tf.keras.Model(inputs=image_input,outputs=x) #(None, 48, 118, 25) 
    return image_input,x
inputs=[]
combine_layers=[]
for _ in range(5):
    
combined=tf.keras.layers.concatenate(axis=1,inputs=[start_of_cnn(),start_of_cnn()])
x=tf.keras.layers.Dense(200, activation="relu")(combined)
x=tf.keras.layers.Dense(200, activation="relu")(x)
x=tf.keras.layers.Dense(1, activation="softmax")(x)
# x=tf.keras.Model(inputs=image_input,outputs=x)